In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd

mclp = pd.read_csv('/content/gdrive/MyDrive/WCRC2023/데이터/최종 데이터/MCLP_결과.csv', encoding='euc-kr')
cctv = pd.read_csv('/content/gdrive/MyDrive/WCRC2023/데이터/최종 데이터/CCTV현황_ver4.csv', encoding='euc-kr')

print(mclp.head())
print(cctv.head())

          위도          경도
0  37.415339  127.427246
1  37.396353  127.327369
2  37.716811  127.307819
3  37.413684  127.223664
4  37.423402  126.666870
                 소재지지번주소         위도          경도  cctvid
0     경기도 의정부시 용현동 391-2  37.737015  127.077234   27732
1        경기도 여주시 가업동 271  37.277578  127.613895   23150
2        경기도 여주시 교동 73-3  37.275391  127.637498   23218
3  경기도 수원시 장안구 송죽동 454-3  37.303568  127.002626   13029
4       경기도 의정부시 호원동 443  37.717627  127.048025   28171


In [ ]:
import pandas as pd
from geopy.distance import geodesic

def find_nearest_cctv(row):
    school_location = (row['위도'], row['경도'])
    distances = cctv.apply(lambda cctv: geodesic(school_location, (cctv['위도'], cctv['경도'])).meters, axis=1)
    nearest_cctv_index = distances.idxmin()
    nearest_cctv = cctv.loc[nearest_cctv_index]
    distance = distances.loc[nearest_cctv_index]
    return pd.Series({'가까운_cctvid': nearest_cctv['cctvid'], '거리': distance})

# apply를 사용하여 각 학교에 대해 가까운 CCTV 찾기
mclp[['가까운_cctvid', '거리']] = mclp.apply(find_nearest_cctv, axis=1)

# 결과 확인
print(mclp)

            위도          경도  가까운_cctvid           거리
0    37.415339  127.427246     23331.0  2138.977650
1    37.396353  127.327369      3155.0  1241.028387
2    37.716811  127.307819      6012.0   830.700628
3    37.413684  127.223664      2928.0   871.936767
4    37.423402  126.666870     15002.0  7098.353372
..         ...         ...         ...          ...
495  37.539186  127.527467     22696.0  2418.529515
496  37.285557  126.909277     35111.0    18.816597
497  37.510964  127.511906     22101.0   772.106506
498  37.575883  126.973154       576.0  8555.354934
499  37.026781  127.234274     18046.0   474.961016

[500 rows x 4 columns]


In [ ]:
# 결과 확인
print(mclp.dtypes)
print(cctv.dtypes)

위도            float64
경도            float64
가까운_cctvid    float64
거리            float64
dtype: object
소재지지번주소     object
위도         float64
경도         float64
cctvid       int64
dtype: object


In [ ]:
# '가까운_cctvid' 컬럼의 데이터타입을 int64로 변경
mclp['가까운_cctvid'] = mclp['가까운_cctvid'].astype('int64')

# 결과 확인
print(mclp.dtypes)

위도            float64
경도            float64
가까운_cctvid      int64
거리            float64
dtype: object


In [ ]:
mclp

,위도,경도,가까운_cctvid,거리
0,37.415339,127.427246,23331,2138.977650
1,37.396353,127.327369,3155,1241.028387
2,37.716811,127.307819,6012,830.700628
3,37.413684,127.223664,2928,871.936767
4,37.423402,126.666870,15002,7098.353372
...,...,...,...,...
495,37.539186,127.527467,22696,2418.529515
496,37.285557,126.909277,35111,18.816597
497,37.510964,127.511906,22101,772.106506
498,37.575883,126.973154,576,8555.354934


In [ ]:
# 두 데이터프레임을 '가까운_cctvid'와 'cctvid'를 기준으로 조인
result_df = pd.merge(mclp, cctv[['cctvid', '소재지지번주소','위도','경도']], left_on='가까운_cctvid', right_on='cctvid', how='left')

# 결과 확인
print(result_df)

          위도_x        경도_x  가까운_cctvid           거리  cctvid  \
0    37.415339  127.427246       23331  2138.977650   23331   
1    37.396353  127.327369        3155  1241.028387    3155   
2    37.716811  127.307819        6012   830.700628    6012   
3    37.413684  127.223664        2928   871.936767    2928   
4    37.423402  126.666870       15002  7098.353372   15002   
..         ...         ...         ...          ...     ...   
495  37.539186  127.527467       22696  2418.529515   22696   
496  37.285557  126.909277       35111    18.816597   35111   
497  37.510964  127.511906       22101   772.106506   22101   
498  37.575883  126.973154         576  8555.354934     576   
499  37.026781  127.234274       18046   474.961016   18046   

                    소재지지번주소       위도_y        경도_y  
0     경기도 여주시 산북면 상품리 145-4  37.403448  127.446260  
1         경기도 광주시 학동리 14-37  37.393174  127.340805  
2    경기도 남양주시 수동면 외방리 552-1  37.724073  127.305541  
3        경기도 광주시 중대동 210-9대  37

In [ ]:
result_df = result_df.rename(columns={'위도_x': '위도_mclp'})
result_df = result_df.rename(columns={'경도_x': '경도_mclp'})
result_df = result_df.rename(columns={'위도_y': '위도_cctv'})
result_df = result_df.rename(columns={'경도_y': '경도_cctv'})
result_df.drop('cctvid', axis=1, inplace=True)
result_df

,위도_mclp,경도_mclp,가까운_cctvid,거리,소재지지번주소,위도_cctv,경도_cctv
0,37.415339,127.427246,23331,2138.977650,경기도 여주시 산북면 상품리 145-4,37.403448,127.446260
1,37.396353,127.327369,3155,1241.028387,경기도 광주시 학동리 14-37,37.393174,127.340805
2,37.716811,127.307819,6012,830.700628,경기도 남양주시 수동면 외방리 552-1,37.724073,127.305541
3,37.413684,127.223664,2928,871.936767,경기도 광주시 중대동 210-9대,37.408438,127.216333
4,37.423402,126.666870,15002,7098.353372,경기도 시흥시 월곶동 1013-5번지,37.388747,126.734255
...,...,...,...,...,...,...,...
495,37.539186,127.527467,22696,2418.529515,경기도 양평군 옥천면 용천리 산87임,37.532295,127.501508
496,37.285557,126.909277,35111,18.816597,경기도 화성시 매송면 송라리 176-2,37.285542,126.909489
497,37.510964,127.511906,22101,772.106506,경기도 양평군 양평읍 공흥리 19-4번지,37.504009,127.511707
498,37.575883,126.973154,576,8555.354934,경기도 고양시 덕양구 덕은동 563-10,37.594558,126.879177


In [ ]:
result_df.to_csv('/content/gdrive/MyDrive/WCRC2023/데이터/최종 데이터/센서위치선정.csv', index=False, encoding='euc-kr')

In [2]:
import pandas as pd

sensor = pd.read_csv('/content/gdrive/MyDrive/WCRC2023/데이터/최종 데이터/센서위치선정.csv', encoding='euc-kr')
sensor

,위도_mclp,경도_mclp,가까운_cctvid,거리,소재지지번주소,위도_cctv,경도_cctv
0,37.415339,127.427246,23331,2138.977650,경기도 여주시 산북면 상품리 145-4,37.403448,127.446260
1,37.396353,127.327369,3155,1241.028387,경기도 광주시 학동리 14-37,37.393174,127.340805
2,37.716811,127.307819,6012,830.700628,경기도 남양주시 수동면 외방리 552-1,37.724073,127.305541
3,37.413684,127.223664,2928,871.936767,경기도 광주시 중대동 210-9대,37.408438,127.216333
4,37.423402,126.666870,15002,7098.353372,경기도 시흥시 월곶동 1013-5번지,37.388747,126.734255
...,...,...,...,...,...,...,...
495,37.539186,127.527467,22696,2418.529515,경기도 양평군 옥천면 용천리 산87임,37.532295,127.501508
496,37.285557,126.909277,35111,18.816597,경기도 화성시 매송면 송라리 176-2,37.285542,126.909489
497,37.510964,127.511906,22101,772.106506,경기도 양평군 양평읍 공흥리 19-4번지,37.504009,127.511707
498,37.575883,126.973154,576,8555.354934,경기도 고양시 덕양구 덕은동 563-10,37.594558,126.879177


In [3]:
sensor = sensor.drop_duplicates(subset='가까운_cctvid', keep='first')
sensor

,위도_mclp,경도_mclp,가까운_cctvid,거리,소재지지번주소,위도_cctv,경도_cctv
0,37.415339,127.427246,23331,2138.977650,경기도 여주시 산북면 상품리 145-4,37.403448,127.446260
1,37.396353,127.327369,3155,1241.028387,경기도 광주시 학동리 14-37,37.393174,127.340805
2,37.716811,127.307819,6012,830.700628,경기도 남양주시 수동면 외방리 552-1,37.724073,127.305541
3,37.413684,127.223664,2928,871.936767,경기도 광주시 중대동 210-9대,37.408438,127.216333
4,37.423402,126.666870,15002,7098.353372,경기도 시흥시 월곶동 1013-5번지,37.388747,126.734255
...,...,...,...,...,...,...,...
494,37.928903,127.122004,7029,1329.633419,경기도 동두천시 걸산동 50-11번지,37.927898,127.106934
495,37.539186,127.527467,22696,2418.529515,경기도 양평군 옥천면 용천리 산87임,37.532295,127.501508
496,37.285557,126.909277,35111,18.816597,경기도 화성시 매송면 송라리 176-2,37.285542,126.909489
497,37.510964,127.511906,22101,772.106506,경기도 양평군 양평읍 공흥리 19-4번지,37.504009,127.511707


In [4]:
sensor.to_csv('/content/gdrive/MyDrive/WCRC2023/데이터/최종 데이터/센서위치선정_ver2.csv', index=False, encoding='euc-kr')